In [ ]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00


In [ ]:
import os
import time
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import tiktoken  # Added tiktoken import
from dataclasses import dataclass
from torch.cuda.amp import autocast, GradScaler  # For mixed precision

# Model Definition and Configurations (Replace with your GPT model code)
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                             .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        return self.c_proj(y)

# Define MLP
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x):
        return self.c_proj(self.gelu(self.c_fc(x)))

# Define Block
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

# Define GPTConfig with the new parameters
@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12    # Set to 12 layers
    n_head: int = 16     # Set to 16 attention heads
    n_embd: int = 1024   # Set embedding size to 1024      # Adjusted embedding dimension

# Define GPT
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        assert T <= self.config.block_size
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)
        x = self.transformer.wte(idx) + self.transformer.wpe(pos)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1)) if targets is not None else None
        return logits, loss

# DataLoader with tiktoken
class DataLoaderLite:
    def __init__(self, B, T, data_path, tokenizer_name="gpt2"):
        self.B = B
        self.T = T
        self.tokenizer = tiktoken.get_encoding(tokenizer_name)
        with open(data_path, 'r', encoding='utf-8') as f:
            text = f.read()
        self.tokens = torch.tensor(self.tokenizer.encode(text), dtype=torch.long)
        self.current_position = 0

    def next_batch(self):
      B, T = self.B, self.T
      end = self.current_position + B * T + 1
      if end > len(self.tokens):
          self.current_position = 0  # Reset to the beginning if we exceed data length
          end = self.current_position + B * T + 1

      buf = self.tokens[self.current_position:end]
      self.current_position += B * T

      # Ensure the buffer length matches the batch size requirements
      if len(buf) < B * T + 1:  # If insufficient data, pad with zeros
          pad_size = (B * T + 1) - len(buf)
          buf = torch.cat([buf, torch.zeros(pad_size, dtype=buf.dtype)], dim=0)

      x = buf[:-1].view(B, T)
      y = buf[1:].view(B, T)
      return x, y


# Initialize Model and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT(GPTConfig())  # Replace with your GPT model initialization
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

# Mixed Precision setup
scaler = GradScaler()

# Number of Parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params:,}")

# Checkpoint and Logs
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("logs", exist_ok=True)
log_file = "logs/training_log.txt"

# DataLoaderLite (Replace with your data loader)
train_loader = DataLoaderLite(B=8, T=256, data_path="/content/input.txt")  # Adjust batch size and sequence length

# Training Loop
max_epochs = 200
extra_epochs = 5
loss_threshold = 0.099999
training_complete = False
loss_history = []
with open(log_file, "w") as log:
    for epoch in range(max_epochs):
        epoch_loss = 0.0
        epoch_accuracy = 0.0  # Initialize total accuracy for the epoch
        batches = tqdm(range(len(train_loader.tokens) // (train_loader.B * train_loader.T)))

        for _ in batches:
            x, y = train_loader.next_batch()
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # Mixed Precision training
            with autocast():
                logits, loss = model(x, y)

            # Calculate accuracy for the batch
            _, predictions = logits.max(dim=-1)  # Get the class with the highest probability
            correct = (predictions == y).float()  # Compare predicted and true classes
            accuracy = correct.sum() / correct.numel()  # Compute the accuracy for the batch

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()  # Accumulate accuracy for the epoch

            batches.set_description(f"Epoch {epoch+1}/{max_epochs} - Loss: {loss.item():.6f} - Accuracy: {accuracy.item()*100:.2f}%")

        # Average Loss and Accuracy for the Epoch
        avg_loss = epoch_loss / len(batches)
        avg_accuracy = epoch_accuracy / len(batches)  # Compute average accuracy for the epoch

        # Log the epoch's loss and accuracy
        log.write(f"Epoch {epoch+1}: Loss = {avg_loss:.6f}, Accuracy = {avg_accuracy*100:.2f}%\n")
        print(f"Epoch {epoch+1} Complete: Average Loss = {avg_loss:.6f}, Average Accuracy = {avg_accuracy*100:.2f}%")

        # Save Model for Even Epochs
        if (epoch + 1) % 2 == 0:
            torch.save(model.state_dict(), f"checkpoints/model_epoch_{epoch+1}.pt")
            print(f"Model saved after Epoch {epoch+1}")

        # Check if Threshold is Met
        if avg_loss < loss_threshold and not training_complete:
            print(f"Loss below threshold ({loss_threshold}). Continuing for {extra_epochs} extra epochs.")
            max_epochs += extra_epochs
            training_complete = True


<ipython-input-2-883d3a1c47f3>:145: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Number of trainable parameters: 203,668,480


  0%|          | 0/165 [00:00<?, ?it/s]<ipython-input-2-883d3a1c47f3>:177: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/200 - Loss: 6.052705 - Accuracy: 17.92%: 100%|██████████| 165/165 [00:47<00:00,  3.48it/s]


Epoch 1 Complete: Average Loss = 6.655091, Average Accuracy = 13.06%


Epoch 2/200 - Loss: 5.513403 - Accuracy: 20.85%: 100%|██████████| 165/165 [00:47<00:00,  3.44it/s]


...

Epoch 172 Complete: Average Loss = 0.091676, Average Accuracy = 97.58%
Model saved after Epoch 172


Epoch 173/205 - Loss: 0.168181 - Accuracy: 95.36%: 100%|██████████| 165/165 [00:48<00:00,  3.41it/s]


Epoch 173 Complete: Average Loss = 0.179909, Average Accuracy = 94.93%


Epoch 174/205 - Loss: 0.181719 - Accuracy: 95.12%:  59%|█████▉    | 97/165 [00:28<00:20,  3.38it/s]


KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download('/content/checkpoints/model_epoch_168.pt')  # Replace with the actual file path


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>